In [1]:
import requests
from bs4 import BeautifulSoup
import os
import time
from keras.utils import get_file
try:
    from urllib.request import urlretrieve
except ImportError:
    from urllib import urlretrieve
import xml.sax

import subprocess
import re
import mwparserfromhell
import json

Using TensorFlow backend.


In [2]:
base_url = 'https://dumps.wikimedia.org/frwiki/'
index = requests.get(base_url).text
soup_index = BeautifulSoup(index, 'html.parser')

# Find the links that are dates of dumps
dumps = [a['href'] for a in soup_index.find_all('a') if 
         a.has_attr('href')]
dumps

['../',
 '20191201/',
 '20191220/',
 '20200101/',
 '20200120/',
 '20200201/',
 '20200220/',
 '20200301/',
 '20200401/',
 'frwiki/',
 'latest/']

In [3]:
dump_url = base_url + '20200301/'
index = requests.get(dump_url).text

In [4]:
soup_dump = BeautifulSoup(index, 'html.parser')

In [5]:
files = []

# Search through all files
for file in soup_dump.find_all('li', {'class': 'file'}):
    text = file.text
    # Select the relevant files
    if 'pages-articles' in text:
        files.append((text.split()[0], text.split()[1:]))
        
files[:5]

[('frwiki-20200301-pages-articles-multistream.xml.bz2', ['4.5', 'GB']),
 ('frwiki-20200301-pages-articles-multistream1.xml-p1p275787.bz2',
  ['471.7', 'MB']),
 ('frwiki-20200301-pages-articles-multistream-index1.txt-p1p275787.bz2',
  ['1.3', 'MB']),
 ('frwiki-20200301-pages-articles-multistream2.xml-p275788p927545.bz2',
  ['517.8', 'MB']),
 ('frwiki-20200301-pages-articles-multistream-index2.txt-p275788p927545.bz2',
  ['3.0', 'MB'])]

In [9]:
files_to_download = [file[0] for file in files if '.xml-p' in file[0] and 'multistream' not in file[0]]
files_to_download[:5]

['frwiki-20200301-pages-articles1.xml-p1p275787.bz2',
 'frwiki-20200301-pages-articles2.xml-p275788p927545.bz2',
 'frwiki-20200301-pages-articles3.xml-p927546p2427545.bz2',
 'frwiki-20200301-pages-articles3.xml-p2427546p2516881.bz2',
 'frwiki-20200301-pages-articles4.xml-p2516882p4016881.bz2']

In [53]:
keras_home = '/users/antoine.bres/.keras/datasets/'
data_paths = []
file_info = []

# Iterate through each file
for file in files_to_download:
    path = keras_home + file
    
    # Check to see if the path exists (if the file is already downloaded)
    if not os.path.exists(path):
        # If not, download the file
        data_paths.append(get_file(file, dump_url + file))
        # Find the file size in MB
        file_size = os.stat(path).st_size / 1e6
        
        # Find the number of articles
        file_articles = int(file.split('p')[-1].split('.')[-2]) - int(file.split('p')[-2])
        file_info.append((file, file_size, file_articles))
        
    # If the file is already downloaded find some information
    else:
        data_paths.append(path)
        # Find the file size in MB
        file_size = os.stat(path).st_size / 1e6
        
        # Find the number of articles
        file_number = int(file.split('p')[-1].split('.')[-2]) - int(file.split('p')[-2])

        file_info.append((file.split('-')[-1], file_size, file_number))

293330944/293324732 [==============================] - 132s 0us/step
